# Brukerhistorie 2
Som bruker ønsker jeg å strømme AR50 slik at jeg kan filtrere data etter behov.  

Funksjonalitet: Det skal være mulig å hente AR50 direkte fra fil. Prosessen omhandler strømming som benytter “partial read" som bruker kan filtrere ut data. 

Akseptkriterier: 
- Jeg skal kunne hente AR50-data direkte fra filer med geografisk filtrering og kolonnefiltrering.  
- Jeg skal kunne filtrere data etter mine egne behov. 

Mål: 
Strømmingen av AR50 skal være en effektiv og brukervennlig prosess. Brukeren skal kunne filtrere data etter egne behov. Prosessen skal også benytte “partial read”, slik at data oppdateres automatisk og ytelsen optimaliseres. 


# Packages

In [0]:
from sedona.spark import *
import requests
import os
import json
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.functions import expr, col, to_timestamp, hour, date_format, lit, to_timestamp, lower, when, desc
from pyspark.sql.types import IntegerType, DoubleType, FloatType, LongType, TimestampType, DateType, StringType
from pyspark.sql import DataFrame
from pyspark.sql.utils import AnalysisException
import pandas as pd
import geopandas as gpd
from shapely import wkb
from datetime import datetime
from shapely.geometry import Point
import geopandas as gpd

# Constant variable

In [0]:
A50_TABLE ="land_techtroll_dev.bronze.ar50"
EPSG = 4326
OPERATOR = ["==", "!=", "<", "<=", ">", ">=", "LIKE"]

# Config
Define dev_config for the development environment and prod_config for the production environment.</br></br>
Each configuration contains:
- catalog_name: the name of the catalog.
- landing_zone_prefix: file path to the "landing zone" where new data is typically placed initially.
- location_prefix: file path to where static data is stored in the cloud.
- static_data_prefix: general file path to static data.

In [0]:
%run ./config

{'catalog_name': 'land_techtroll_dev',
 'landing_zone_prefix': '/Volumes/land_techtroll_dev/external_dev/landing_zone',
 'location_prefix': '/Volumes/land_techtroll_dev/external_dev/static_data/cloudFiles',
 'static_data_prefix': '/Volumes/land_techtroll_dev/external_dev/static_data',
 'env': 'dev'}

# Set catelog
In this notebook, the catalog is set to `land_techtroll_dev`, as this is where the relevant databases are located. Setting the catalog ensures that all subsequent queries reference the correct data environment, making it easier to access and manage the necessary tables for our testing and development.


In [0]:
spark.sql(f'USE CATALOG {spark.conf.get("conf.catalog_name")}')

DataFrame[]

# Get tabell from database
The database follows the Medallion Architecture, which organizes data into layers:
- Bronze: Raw or minimally processed data (used in this case).
- Silver: Cleaned and structured data.
- Gold: Aggregated, business-ready data

This notebook using only bronze layer to work with raw data for early-stage testing and development.

Important note! <i> Spark table has geometry data as binary, hence it need to be convert</i>

In [0]:
# Read AIS data from database
AR50_DF = spark.table(A50_TABLE)

# Clean the table
Remove unnecessary data table in following columns:
- Arealtype
- Jordbruk
- Skogbonitet
- Treslag
- vegetasjonsdekke

The following values is removed:
- 99: "Ikke kartlagt"
- 98: "Ikke definert"

In [0]:
# If there are more values that needs be remove, add them here
null_values = [98, 99]

AR50_DF = AR50_DF.withColumn(
    "vegetasjonsdekke",
    when(col("vegetasjonsdekke").isin(null_values), None).otherwise(col("vegetasjonsdekke"))
)

AR50_DF = AR50_DF.withColumn(
    "Arealtype",
    when(col("Arealtype").isin(null_values), None).otherwise(col("Arealtype"))
)

AR50_DF = AR50_DF.withColumn(
    "Treslag",
    when(col("Treslag").isin(null_values), None).otherwise(col("Treslag"))
)

AR50_DF = AR50_DF.withColumn(
    "Skogbonitet",
    when(col("Skogbonitet").isin(null_values), None).otherwise(col("Skogbonitet"))
)

AR50_DF = AR50_DF.withColumn(
    "Jordbruk",
    when(col("Jordbruk").isin(null_values), None).otherwise(col("Jordbruk"))
)

# Checks methods
Various methods for check if the variable, dataframe or so on, it correct

In [0]:
def check_df_columns(df: DataFrame):
    """
    Validates that the provided DataFrame contains at least one column.

    Parameters:
    df (DataFrame): The DataFrame to validate.

    Raises:
    ValueError: If the DataFrame has no columns.
    """
    if not df.columns:
        # Raise an error if the DataFrame has no columns
        raise ValueError("DataFrame has no columns to search.")

def check_df_contains_column(df: DataFrame, required_cols: list):
    """
    Checks whether the given DataFrame contains all the specified required columns.

    Parameters:
    df (DataFrame): The DataFrame to validate.
    required_cols (list): A list of column names that are expected in the DataFrame.

    Raises:
    ValueError: If one or more required columns are missing. 
                The error message will include the names of the missing columns.
    """
    # Create a list of missing columns by checking which required columns are not in the DataFrame
    missing = [col for col in required_cols if col not in df.columns]

    # Raise an error if any required columns are missing
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
def is_column_string(df: DataFrame, column_name: str) -> bool:
    """
    Checks whether the specified column in the DataFrame has a string data type.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if the column is of type StringType, False otherwise.
    """
    # Access the column's data type from the DataFrame schema and check if it's a StringType
    return isinstance(df.schema[column_name].dataType, StringType)
  
def is_column_integer(df: DataFrame, column_name: str) -> bool:
    """
    Checks if the specified column is of an integer-compatible type.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if the column is IntegerType or LongType, otherwise False.
    """
    # Check if the column's data type is either IntegerType or LongType
    return isinstance(df.schema[column_name].dataType, (IntegerType, LongType))

def is_column_double(df: DataFrame, column_name: str) -> bool:
    """
    Checks if the specified column is of a floating-point type.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if the column is DoubleType or FloatType, otherwise False.
    """
    # Check if the column's data type is either DoubleType or FloatType
    return isinstance(df.schema[column_name].dataType, (DoubleType, FloatType))


def is_column_datetime(df: DataFrame, column_name: str) -> bool:
    """
    Checks if the specified column is a timestamp or date.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if the column is TimestampType or DateType, otherwise False.
    """
    # Check if the column's data type is either TimestampType or DateType
    return isinstance(df.schema[column_name].dataType, (TimestampType, DateType))


# Filter methods
Various methods are available for filtering the data based on specific conditions or criteria.

In [0]:
def search_df(df: DataFrame, search_term: str, limit: int = 100) -> DataFrame:
    """
    Searches a DataFrame for rows where any column contains the given search term (case-insensitive).

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    search_term (str): The term to search for within all columns.
    limit (int): Maximum number of rows to return. Default is 100.

    Returns:
    DataFrame: A DataFrame with up to `limit` rows where at least one column contains the search term.

    Raises:
    ValueError: If the DataFrame has no columns or no matches are found.
    """
    # Ensure the DataFrame has columns to search
    check_df_columns(df)

    filtered_dfs = []

    # Convert search term to lowercase once
    search_term_lower = search_term.lower()

    # Search each column for the search term (case-insensitive)
    for col_name in df.columns:
        filtered_col_df = df.filter(lower(col(col_name).cast("string")).contains(search_term_lower))

        if filtered_col_df.take(1):  # Efficient check for matches
            filtered_dfs.append(filtered_col_df)

    if not filtered_dfs:
        raise ValueError("No matches found for the given search term.")

    # Combine all matching rows and remove duplicates
    combined_filtered_df = filtered_dfs[0]
    for filtered_df in filtered_dfs[1:]:
        combined_filtered_df = combined_filtered_df.union(filtered_df)

    return combined_filtered_df.distinct().limit(limit)

def select_columns(df: DataFrame, columns: list, limit: int = 100) -> DataFrame:
    """
    Selects specific columns from a DataFrame and returns up to `limit` rows.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    columns (list): A list of column names to select.
    limit (int): The maximum number of rows to return. Default is 100.

    Returns:
    DataFrame: A DataFrame containing only the selected columns, limited to `limit` rows.

    Raises:
    ValueError: If any of the specified columns are missing from the DataFrame.
    """
    # Validate that all specified columns exist in the DataFrame
    check_df_contains_column(df, columns)

    # Select specified columns and limit to the desired number of rows
    return df.select(*columns).limit(limit)

In [0]:
def operation_on_column(df: DataFrame, op: str, column: str, value: dict) -> DataFrame | None:
    """
    Applies a dynamic filter operation on a specified column in the DataFrame.

    Parameters:
    df (DataFrame): The input PySpark DataFrame.
    op (str): The operator to use for filtering (e.g., '>=', '==', 'between').
    column (str): The name of the column to apply the filter to.
    value (dict): A dictionary containing the 'search' value(s).
    type_value (str): A type hint (e.g., 'hour', 'date') to handle special filtering cases.

    Returns:
    DataFrame | None: A new filtered DataFrame if the operator is valid, otherwise None.
    """
    # Extract the search value or value range from the filter definition
    search = value["search"]
    tmp_df = None

    # Handle standard comparison operators
    if op == ">=":
        tmp_df = df.filter(col(column) >= search)
    elif op == "<=":
        tmp_df = df.filter(col(column) <= search)
    elif op == ">":
        tmp_df = df.filter(col(column) > search)
    elif op == "<":
        tmp_df = df.filter(col(column) < search)
    elif op == "==":
        tmp_df = df.filter(col(column) == search)

    # Handle range-based filtering (e.g., 'between')
    elif op == "between":
        # Generic between filter for numeric, date, or timestamp values
        tmp_df = df.filter(col(column).between(search[0], search[1]))

    return tmp_df
def filter_dataframe(df: DataFrame, filters: dict, limit: int = 100) -> DataFrame:
    """
    Dynamically filters a DataFrame based on a dictionary of column filter definitions.

    Parameters:
    - df (DataFrame): The input DataFrame to filter.
    - filters (dict): Dictionary of filters with structure:
        {
            "column_name": {
                "operator": "==", ">=", "<", "between", ...
                "value": {
                    "search": value or (start, end)
                }
            }
        }
    - limit (int): Maximum number of rows to return.

    Returns:
    - DataFrame: Filtered DataFrame with the applied filters.
    """
    # Validate all columns exist
    check_df_contains_column(df, filters.keys())

    filter_df = df

    for col_name, value in filters.items():
        tmp_df = operation_on_column(
            df,
            value["operator"],
            col_name,
            value["value"]
        )

        if tmp_df is None:
            raise ValueError(f"Invalid operator {value['operator']} for column '{col_name}'")

        # Use intersection logic instead of join to ensure rows match all filters
        filter_df = filter_df.intersect(tmp_df)

    return filter_df.limit(limit)


# Gis query

In [0]:
def create_buffer_radius(lon, lat, radius_m=500):
    """
    Create a buffer area (radius in meters) around a given longitude and latitude.

    Args:
        lon (float): Longitude of the center point.
        lat (float): Latitude of the center point.
        radius_m (int, optional): Radius of the buffer in meters. Defaults to 500.

    Returns:
        DataFrame: A Spark DataFrame containing the buffer geometry as GeoJSON.
    """
    # Create a DataFrame with a single point (id, longitude, latitude)
    data = [(1, lon, lat)]
    df_raw = spark.createDataFrame(data, ["id", "lon", "lat"])
    df_raw.createOrReplaceTempView("geom_points")

    # Convert the longitude and latitude into a geometry point with SRID 3857
    geom_df = spark.sql("""
    SELECT 
        id,
        ST_SetSRID(ST_Point(lon, lat), 3857) AS geometry
    FROM geom_points
    """)

    # Create a buffer around the point and transform it, returning it as GeoJSON
    query = f"""
        SELECT 
            id,
            st_asgeojson(
                ST_Transform(
                    ST_SetSRID(
                        ST_Buffer(
                            ST_SetSRID(ST_Point(lon, lat), {EPSG}),
                            {radius_m}, True
                        ),
                        {EPSG}
                    ),
                    'EPSG:{EPSG}'
                )
            ) AS geom
        FROM geom_points
    """
    buffer_df = spark.sql(query)
    return buffer_df


def get_geomtry_within(df, lon, lat, radius_m=500):
    """
    Find geometries within a given radius from a specified point.

    Args:
        df (DataFrame): Spark DataFrame containing geometries to query against.
        lon (float): Longitude of the center point.
        lat (float): Latitude of the center point.
        radius_m (int, optional): Search radius in meters. Defaults to 500.

    Returns:
        Tuple[DataFrame, DataFrame]: 
            - DataFrame with geometries that intersect the buffer area.
            - DataFrame with the buffer geometry itself.
    """
    # Create a buffer around the given point
    buffer_df = create_buffer_radius(lon, lat)
    
    # Register the input DataFrame as a temporary view
    df.createOrReplaceTempView("Within_view")

    # Query to select geometries that intersect the buffer
    query = f"""
        SELECT
            gml_Id,
            Treslag,
            geometry
        FROM Within_view
        WHERE ST_Intersects(
            ST_GeomFromText(geometry),
            (SELECT 
                ST_Transform(
                    ST_SetSRID(
                        ST_Buffer(
                            ST_SetSRID(ST_Point(lon, lat), {EPSG}),
                            {radius_m}, True
                        ),
                        {EPSG}
                    ),
                    'EPSG:{EPSG}'
                ) AS geom
            FROM geom_points)
        )
    """
    within_df = spark.sql(query)
    return within_df, buffer_df


# Map visualization


In [0]:
def prepare_for_kepler(df: DataFrame) -> DataFrame:
    """
    Cleans a DataFrame for visualization in SedonaKepler.
    
    - Converts DecimalType to FloatType
    - Converts DateType to String ("yyyy-MM-dd")

    Args:
        df (DataFrame): The input PySpark DataFrame

    Returns:
        DataFrame: Cleaned DataFrame ready for Kepler.gl
    """
    # Convert decimal columns to float
    decimal_cols = [f.name for f in df.schema.fields if "decimal" in str(f.dataType).lower()]
    for col_name in decimal_cols:
        df = df.withColumn(col_name, col(col_name).cast("float"))

    # Convert date columns to string (yyyy-MM-dd)
    if "date" in df.columns:
        df = df.withColumn("date", date_format(col("date"), "yyyy-MM-dd"))

    return df
    
def visualize_map(filter_df: DataFrame, name: str = None):
    """
    Visualizes a filtered PySpark DataFrame on a Kepler-style interactive map.

    Parameters:
    filter_df (DataFrame): The filtered DataFrame containing spatial data (geometry or coordinates).
    name (str, optional): A name or label to assign to the map layer. Defaults to None.

    Returns:
    None: Displays the interactive map in the notebook environment.
    """
    # Preprocess the DataFrame to make it compatible with Kepler visualization
    filtered_df_clean = prepare_for_kepler(filter_df)

    # Create an empty interactive map using SedonaKepler
    map = SedonaKepler.create_map()

    # Add the cleaned DataFrame to the map, with an optional layer name
    SedonaKepler.add_df(map, filtered_df_clean, name=name)

    # Display the map in the notebook
    return map

# Input functions

In [0]:
def choose_col():
    print("\nFølgende kolumner er tilgjengelig")
    for vailed_col in AR50_DF.columns:
        print(vailed_col)
    col = input("""Hilken kolumne ønsker du filtere etter?""")
    return col if col in AR50_DF.columns else None

def value_converter(col:str):
    try:
        value = input(f"Skriv inn verdien du ønsker flitere etter")
        if col == "arealtype" or col == "lokalId" or col == "Jordbruk" or col == "områdeId" or col == "Skogbonitet" or col == "Treslag" or col == "vegetasjonsdekke":
            print(value)
            return int(value)
        elif col == "Kopiddato" or col == "Oppdateringsdato":
            return datetime.strptime(value, "%Y-%m-%d %H:%M:%S")
        else:
            return value
    except ValueError:
        print(f"Du har skrev ikke en gylding input for {col}, prøv igjen")
        return None
    
def choose_operator():
    print("Hvilken operator ønsker du å bruke?")
    print("Følgende operatorer er tilgjengelige:")
    print("==    sjekker om to verdier er like")
    print("!=    sjekker om to verdier er ulike")
    print(">     sjekker om en verdi er større enn en annen")
    print("<     sjekker om en verdi er mindre enn en annen")
    print(">=    sjekker om en verdi er større enn eller lik en annen")
    print("<=    sjekker om en verdi er mindre enn eller lik en annen")
    print("LIKE  sjekker om en verdi inneholder en spesifikk tekst (brukes vanligvis med tekststrenger)")

    operator = input()
    return operator if operator in ["==", "!=", ">", "<", ">=", "<=", "LIKE"] else None

# Optimization

In [0]:
def difference_between_rows(df: DataFrame, filter_df: DataFrame) -> int:
    """
    Calculates and prints the difference in row counts between the original and filtered DataFrames.

    Parameters:
    df (DataFrame): The original (unfiltered) DataFrame.
    filter_df (DataFrame): The filtered version of the DataFrame.

    Returns:
    int: The number of rows removed during filtering.
    """
    # Count rows in both DataFrames
    original_count = df.count()
    filtered_count = filter_df.count()

    # Print summary
    print(f"The original DataFrame has {original_count} rows.")
    print(f"The filtered DataFrame has {filtered_count} rows.")
    print(f"The difference is {original_count - filtered_count} rows.")

    # Return the difference in row count
    return original_count - filtered_count

In [0]:
def difference_between_size(original_table: str, filter_df: DataFrame) -> float:
    """
    Compares the disk size (in MB) of the original Delta table and a filtered DataFrame.

    This function:
    - Writes the filtered DataFrame to a temporary table
    - Compares table sizes using DESCRIBE DETAIL
    - Cleans up the temporary table afterward
    - Returns the size difference in megabytes

    Parameters:
    original_table (str): The name of the original Delta table (e.g., 'bronze.ais').
    filter_df (DataFrame): The filtered DataFrame to compare.

    Returns:
    float: The difference in size (original - filtered) in megabytes.
    """
    tmp_table = "bronze.ais_tmp"

    try:
        # Save the filtered DataFrame as a temporary table
        filter_df.write.mode("overwrite").saveAsTable(tmp_table)

        # Get size of the original table
        original_info = spark.sql(f"DESCRIBE DETAIL {original_table}")
        original_size_bytes = original_info.select("sizeInBytes").collect()[0][0]
        original_size_mb = original_size_bytes / (1024 * 1024)

        # Get size of the temporary (filtered) table
        tmp_info = spark.sql(f"DESCRIBE DETAIL {tmp_table}")
        tmp_size_bytes = tmp_info.select("sizeInBytes").collect()[0][0]
        tmp_size_mb = tmp_size_bytes / (1024 * 1024)

        # Calculate and print the size difference
        size_diff_mb = original_size_mb - tmp_size_mb
        print(f"Original table size: {original_size_mb:.2f} MB")
        print(f"Filtered table size: {tmp_size_mb:.2f} MB")
        print(f"Difference in size: {size_diff_mb:.2f} MB")

        return size_diff_mb

    except AnalysisException as e:
        print(f"[ERROR] Table not found or query failed: {e}")
        return 0.0
    except Exception as e:
        print(f"[ERROR] Unexpected error: {e}")
        return 0.0
    finally:
        # Always drop the temporary table, even if something fails
        try:
            spark.sql(f"DROP TABLE IF EXISTS {tmp_table}")
            print(f"Temporary table '{tmp_table}' dropped.")
        except Exception as drop_err:
            print(f"[WARNING] Could not drop temporary table: {drop_err}")


# Save filtrered df

In [0]:
def save_filtered_df(
    filtered_df: DataFrame,
    filename: str,
    path: str = "/Workspace/Users/andrine.flatby@kartverket.no/techtroll-data-ingestor/src/databricks/nedlastet_filer",
    file_format: str = "csv",
    include_timestamp: bool = False
) -> None:
    """
    Saves a PySpark DataFrame to disk as CSV, Parquet, or JSON, with optional timestamp in filename.

    Parameters:
    filtered_df (DataFrame): The filtered PySpark DataFrame to save.
    filename (str): Base name for the file (without extension).
    path (str): Directory path where the file will be saved.
    file_format (str): File format: 'csv', 'parquet', or 'json'. Default is 'csv'.
    include_timestamp (bool): If True, adds a timestamp to the filename.

    Returns:
    None
    """
    try:
        # Convert to pandas DataFrame
        pandas_df = filtered_df.toPandas()

        # Validate format
        file_format = file_format.lower()
        if file_format not in {"csv", "parquet", "json"}:
            raise ValueError(f"Unsupported file format: {file_format}")

        # Optional timestamp
        if include_timestamp:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{filename}_{timestamp}"

        # Construct full file path
        full_path = os.path.join(path, f"{filename}.{file_format}")

        # Ensure path exists
        os.makedirs(path, exist_ok=True)

        # Save based on format
        if file_format == "csv":
            pandas_df.to_csv(full_path, index=False)
        elif file_format == "parquet":
            pandas_df.to_parquet(full_path, index=False)
        elif file_format == "json":
            pandas_df.to_json(full_path, orient="records", lines=True)

        print(f"[INFO] File saved to: {full_path}")

    except Exception as e:
        print(f"[ERROR] Failed to save DataFrame: {e}")

# User case examples

## Visning av treslag innenfor et gitt område 

Som arealplanlegger i Kristiansand kommune ønsker jeg å hente ut og analysere AR50-data innenfor en radius på 500-meter rundt et valgt punkt, slik at jeg kan finne hvilke treslag som finnes i området og vurdere grøntstruktur i byplanlegging. 

Akseptkriterier: 
- Bruker skal kunne angi koordinater og buffer-radius 
- Systemet skal returnere forekomster av treslag innenfor området 
- Treslagene skal vises både i tabell og på kart 
<br>

Input:
- Koordinater (f.eks. 8.003180405023354, 58.16368151324909)  
- En radius i meter (f.eks. 500) 

Output:
- Tabell 
- Kart 


In [0]:
while True: 
    print("""Hilken kolumne ønsker du filtere etter?""")
    print("\nFølgende kolumner er tilgjengelig")
    for vailed_col in AR50_DF.columns:
        print(vailed_col)
    col = input()
    if col in AR50_DF.columns:
        while True: 
            try:
                print("""Skriv inn koordinatene din:""")
                lat = float(input("latitude"))
                lon = float(input("longitude"))
                while True: 
                    try:
                        print("""Skriv inn radiusen i meter du ønsker søke i:""")
                        radius = int(input("radius"))
                    except ValueError:
                        print("Du har skrev ikke en gylding radius, prøv igjen")
                        continue
                    break
            except ValueError:
                print("Du har skrev ikke en gylding koordinat, prøv igjen")
                continue
            break
        break
    else:
        print("Du har skrev ikke en gylding kolumne, prøv igjen")

filtered_df = AR50_DF.filter(AR50_DF[col].isNotNull())
df, buffer_df = get_geomtry_within(filtered_df,lon, lat, radius)
df.display()
                        
map_df = SedonaKepler.create_map()

# Add the cleaned DataFrame to the map, with an optional layer name
map_df.add_data(df.toPandas(), "points")
map_df.add_data(buffer_df.toPandas(), "buffer")
map_df




Hilken kolumne ønsker du filtere etter?

Følgende kolumner er tilgjengelig
gml_Id
Arealtype
lokalId
Navnerom
Informasjon
Jordbruk
Kartstandard
Kopiddato
områdeId
OriginalDatavert
Oppdateringsdato
Skogbonitet
Treslag
vegetasjonsdekke
geometry


 Treslag

Skriv inn koordinatene din:


latitude 58.16368151324909

longitude 8.003180405023354

Skriv inn radiusen i meter du ønsker søke i:


radius 500

gml_Id Treslag geometry idb498c719-7486-4106-a629-170094d23aea 33 POLYGON ((8.005060930372247 58.16038908892208, 8.004694602276352 58.16030131853006, 8.004594471461935 58.16016062486779, 8.004638427238715 58.15993775156891, 8.004753531088042 58.159786386628674, 8.004939782878553 58.15970652985068, 8.004915196674213 58.15961505734768, 8.004562063376559 58.15946042472104, 8.004195746032746 58.15937265311437, 8.003976119068616 58.159405610778336, 8.003781074836645 58.159530040716795, 8.003599217660945 58.15958760847904, 8.003430548653366 58.15957831414517, 8.003401570182152 58.15950912870705, 8.00351228154379 58.15938005224562, 8.003454324648994 58.15924168140114, 8.003227701524724 58.159094015985026, 8.00297470175735 58.15908007373783, 8.00269532254258 58.15919985444291, 8.002471298681503 58.159255096737425, 8.00230263165497 58.15924580094561, 8.002227092045759 58.15919657852954, 8.002244679696407 58.15910742965219, 8.002388766683833 58.15902525265799, 8.002659352250388 58.158950047100035, 8.002850000472105 58.15884790614587, 8.002960711560542 58.15871883016032, 8.002936130977277 58.15862735739207, 8.002776259717733 58.158573487783215, 8.002751679684929 58.15848201499082, 8.002862389883486 58.15835293907259, 8.002795644113256 58.15825914251318, 8.002551443623798 58.15820062505877, 8.002378384897538 58.15821361670715, 8.002276466939083 58.15829811772873, 8.002103407662398 58.15831110899048, 8.001859208062852 58.158252590222304, 8.00146652685728 58.158298534340105, 8.000925360075724 58.15844893988195, 8.00066615622099 58.15868247627651, 8.000688918783924 58.15899914532545, 8.000611569234072 58.159175117958114, 8.00043410717324 58.15921039398269, 8.000285616887409 58.159314855774866, 8.000166097710643 58.15948850342253, 8.000290783180878 58.15972067332103, 8.00065967867394 58.1600113648633, 8.00083533272679 58.16020128457713, 8.000817740803235 58.16029043316978, 8.000888882650619 58.16036194358018, 8.001048758718325 58.1604158156685, 8.001119901273103 58.160487325964226, 8.00110230986555 58.16057647460724, 8.001173452828139 58.16064798488153, 8.001333330610278 58.160701856647236, 8.001475618229101 58.16084487693247, 8.001600316458656 58.1610770458128, 8.00183573966836 58.16118013985719, 8.002181886385456 58.161154158418256, 8.002405922865336 58.16109891689906, 8.002507849065344 58.16101441594552, 8.00268531843613 58.16097913717347, 8.00293833152551 58.16099308027723, 8.003140382617055 58.161049273661064, 8.003291472156906 58.16114771757783, 8.003316054918631 58.161239190323755, 8.003214129908311 58.161323691951516, 8.003238712557437 58.16141516471016, 8.003389803859948 58.16151360854688, 8.00345216012476 58.16162969221184, 8.003425780551465 58.161763415819095, 8.003572477258452 58.16188414674772, 8.003892251905587 58.161991884432254, 8.003979195514365 58.16219944056425, 8.0038333045666 58.16250681554888, 8.003860491842143 58.1628011973408, 8.004060761650072 58.16308258591895, 8.004296205212743 58.16318567582528, 8.004566819419827 58.16311046679865, 8.00477506954098 58.16291917404626, 8.004920954377466 58.16261179785018, 8.005036066568222 58.16246043282719, 8.005120408168288 58.16246507910376, 8.005129199506323 58.16242050441223, 8.005062440815278 58.16232670877319, 8.005071232178496 58.16228213408622, 8.005155573363012 58.162286780330696, 8.005215326301169 58.1621999540307, 8.005250490658485 58.16202165521397, 8.005188127854055 58.16190557229558, 8.005028238573356 58.161851705146084, 8.004999255437504 58.161782519974516, 8.005101177705154 58.161698016855084, 8.005038815883658 58.16158193388407, 8.004812171746105 58.16143427085637, 8.004707641866048 58.16131586454643, 8.004725224762383 58.16122671522493, 8.004869315189021 58.16114453548515, 8.0052752105694 58.161031719352565, 8.00494843504939 58.16074336335234, 8.004894866178539 58.160582705665426, 8.005060930372247 58.16038908892208)) ida658c04f-7ffd-449e-a4ec-b6754b06c2a2 31 POLYGON ((7.997502940783783 58.168443637533315, 7.997782411777257 58.16832387018386, 7.9978465949786 58.168

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'points': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8], 'columns': ['gml_Id', 'Treslag', 'geometry'], …


## Analyse av arealtype 10 (bebyggelse) i Kristiansand 

Brukerhistorie 

Som arealplanlegger i Kristiansand kommune ønsker jeg å hente ut og analysere AR50-data innenfor en radius på 500-meter rundt et valgt punkt, slik at jeg kan finne hvilke treslag som finnes i området og vurdere grøntstruktur i byplanlegging. 

Akseptkriterier: 
- Bruker skal kunne spesifisere områdekode og arealtype.  
- Systemet skal returnere de 10 største forekomstene basert på areal.  

Input 
- En områdekode (f.eks. 4204) 
- En arealtype (f.eks. 10) 

Output: 

- Tabell med topp 10 forekomster  
- Kart med markering av disse områdene  

 



In [0]:
filter_obj = {}
while True:
  col = choose_col()
  if col == None:
    print("Du har skrev ikke en gylding kolumne, prøv igjen")
    continue

  while True:
    operator = choose_operator()
    if operator == None:
      print("Du har skrev ikke en gylding operatoren, prøv igjen")
      continue
    break

  while True:
    value = value_converter(col)
    if value == None:
      print("Du har skrev ikke en gylding input, prøv igjen")
      continue
    break
  filter_obj[col] = {  # Sett en ny nøkkel i dict direkte
    "operator": operator,
    "value": {
        "search": f"{value}"
    }
  }


  done = input("Ønker du å flitere etter noe med? ja eller nei").lower()
  if done == "ja":
    continue
  elif done == "nei":
    break
  else:
    print("Du har skrev ikke en gyldig svar, prøv igjen")

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
filtered_df =  filter_dataframe(AR50_DF, filter_obj)
filtered_df.display()

map_user2 = SedonaKepler.create_map()

# Add the cleaned DataFrame to the map, with an optional layer name
map_user2.add_data(filtered_df.toPandas(), "filtered")
map_user2

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data


## Henting av arealtyper innenfor buffer rundt koordinat 

Brukerhistorie 

Som miljøkonsulent ønsker jeg å kunne angi koordinater med buffer for å se hvilke typer vegetasjon eller arealtyper som finnes innenfor det avgrensede området, slik at jeg kan vurdere påvirkning på naturmiljøet i et utbyggingsprosjekt. 

Akseptkriterier: 
- Bruker skal kunne angi koordinater og buffer-radius.  
- Systemet skal returnere en oversikt over forekomster av ulike arealtyper.  
- Brukeren skal kunne se resultatene både som liste og visuelt på kart. 

Input 
- Koordinater (f.eks. 58.1467, 7.9956)  
- En radius i meter (f.eks. 1000) 

Output: 
- Tabell med arealtyper  
- Kart med arealfordeling i området 


In [0]:
while True: 
    print("""Hilken kolumne ønsker du filtere etter?""")
    print("\nFølgende kolumner er tilgjengelig")
    for vailed_col in AR50_DF.columns:
        print(vailed_col)
    col = input()
    if col in AR50_DF.columns:
        while True: 
            try:
                print("""Skriv inn koordinatene din:""")
                lat = float(input("latitude"))
                lon = float(input("longitude"))
                while True: 
                    try:
                        print("""Skriv inn radiusen i meter du ønsker søke i:""")
                        radius = int(input("radius"))
                    except ValueError:
                        print("Du har skrev ikke en gylding radius, prøv igjen")
                        continue
                    break
            except ValueError:
                print("Du har skrev ikke en gylding koordinat, prøv igjen")
                continue
            break
        break
    else:
        print("Du har skrev ikke en gylding kolumne, prøv igjen")

filtered_df = AR50_DF.filter(AR50_DF[col].isNotNull())
df, buffer_df = get_geomtry_within(filtered_df,lon, lat, radius)
df.display()
                        
map_df = SedonaKepler.create_map()

# Add the cleaned DataFrame to the map, with an optional layer name
map_df.add_data(df.toPandas(), "points")
map_df.add_data(buffer_df.toPandas(), "buffer")
map_df




com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data